# Preparation

In [6]:
# Install dependencies
%pip install -q ipywidgets transformers tqdm
%pip install -q seqeval
%pip install -q accelerate
%pip install -q transformers[torch]

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


## Set Seed and CUDA

In [3]:
import torch
import torchtext

SEED = 1234
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

print("PyTorch Version: ", torch.__version__)
print("torchtext Version: ", torchtext.__version__)
print(f"Using {'GPU' if str(DEVICE) == 'cuda' else 'CPU'}.")

PyTorch Version:  2.2.1+cu121
Using GPU.


## Data Prep

### Download the Dataset
this will download the huggingface dataset ready for use

In [4]:
from datasets import load_dataset

dataset = load_dataset("surrey-nlp/PLOD-filtered")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset plo_dfiltered_config downloaded and prepared to /root/.cache/huggingface/datasets/surrey-nlp___plo_dfiltered_config/PLODfiltered/0.0.2/bd402e453833e246144db398363ec772d4710a494fc4de15e48c7b1ddac6b82b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

### Get Label List
this gets the list of labels for the dataset

In [5]:
label_list = dataset["train"].features[f"ner_tags"].feature.names
print(label_list)

['B-O', 'B-AC', 'I-AC', 'B-LF', 'I-LF']


### Split The Set Into Train, Val and Test Sets

In [6]:
train = dataset['train']
print(f"train size: {len(train)}")
val = dataset['validation']
print(f"train size: {len(val)}")
test = dataset['test']
print(f"train size: {len(test)}")


train size: 112652
train size: 24140
train size: 24140


### Display Random Samples
this code displays some random samples from the train set

In [8]:
import pandas as pd
from datasets import ClassLabel, Sequence
import random
from IPython.display import display, HTML
import matplotlib.pyplot as plt
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))


show_random_elements(train)

,id,tokens,pos_tags,ner_tags
0,85508,"[Comparison, of, different, cutpoints, and, conditions, for, the, receiver, operating, characteristic, (, ROC, ), curve, predicting, MPNST, vs., plexiform, neurofibroma, clinical, status, .]","[NOUN, ADP, ADJ, NOUN, CCONJ, NOUN, ADP, DET, NOUN, VERB, ADJ, PUNCT, PROPN, PUNCT, NOUN, VERB, PROPN, ADP, ADJ, NOUN, ADJ, NOUN, PUNCT]","[B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-LF, I-LF, I-LF, B-O, B-AC, B-O, B-O, B-O, B-AC, B-O, B-O, B-O, B-O, B-O, B-O]"
1,98192,"[In, S., cerevisiae, ,, the, ubiquitin, -, like, protein, Atg8, is, C, -, terminally, conjugated, to, the, phospholipid, phosphatidylethanolamine, (, PE, ), to, generate, Atg8, -, PE, ., During, autophagosome, formation, ,, Atg8, -, PE, is, cleaved, by, Atg4, to, release, delipidated, Atg8, (, Atg8G116, ), and, PE, ., Although, delipidation, of, Atg8, -, PE, is, important, for, autophagosome, formation, ,, it, remains, controversial, whether, the, delipidation, reaction, is, required, for, targeting, of, Atg8, to, the, VICS, or, for, subsequent, IM, expansion, .]","[ADP, PROPN, NOUN, PUNCT, DET, NOUN, PUNCT, ADJ, NOUN, PROPN, AUX, NOUN, PUNCT, ADV, VERB, ADP, DET, NOUN, NOUN, PUNCT, PROPN, PUNCT, ADP, VERB, PROPN, PUNCT, PROPN, PUNCT, ADP, NOUN, NOUN, PUNCT, PROPN, PUNCT, PROPN, AUX, VERB, ADP, PROPN, ADP, VERB, VERB, PROPN, PUNCT, NOUN, PUNCT, CCONJ, PROPN, PUNCT, SCONJ, NOUN, ADP, PROPN, PUNCT, PROPN, AUX, ADJ, ADP, NOUN, NOUN, PUNCT, PRON, VERB, ADJ, SCONJ, DET, NOUN, NOUN, AUX, VERB, ADP, NOUN, ADP, PROPN, ADP, DET, PROPN, CCONJ, ADP, ADJ, PROPN, NOUN, PUNCT]","[B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-LF, B-O, B-AC, B-O, B-O, B-O, B-O, B-O, B-AC, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-AC, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-AC, B-O, B-O, B-O, B-O, B-O, B-O, B-AC, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-AC, B-O, B-O]"
2,77088,"[Moreover, ,, tissue, -, specific, rescue, experiments, indicate, that, the, circadian, clock, component, PERIOD, (, PER, ), [, 7, ], and, the, circadian, output, ion, channel, NARROW, ABDOMEN, (, NA, ), [, 16, ], are, each, required, in, the, PDF+, LNv, to, promote, robust, ,, sustained, DD, rhythmicity, .]","[ADV, PUNCT, NOUN, PUNCT, ADJ, NOUN, NOUN, VERB, SCONJ, DET, ADJ, NOUN, NOUN, NOUN, PUNCT, PROPN, PUNCT, X, X, X, CCONJ, DET, ADJ, NOUN, NOUN, NOUN, NOUN, NOUN, PUNCT, PROPN, PUNCT, X, X, X, AUX, PRON, VERB, ADP, DET, PROPN, PROPN, PART, VERB, ADJ, PUNCT, ADJ, PROPN, NOUN, PUNCT]","[B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-LF, B-O, B-AC, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-LF, I-LF, B-O, B-AC, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-AC, B-O, B-O, B-O, B-O, B-O, B-O, B-AC, B-O, B-O]"
3,23937,"[The, last, study, found, that, a, model, for, predicting, major, renal, events, using, eGFR, and, albumin, /, creatinine, ratio, (, ACR, ), (, AUC, :, 0.818, ), was, superior, to, models, with, either, of, the, predictors, alone, (, AUC, :, 0.779, for, eGFR, ,, and, 0.752, for, ACR, ), ;, all, three, models, were, inferior, to, an, expanded, model, with, five, additional, variables, (, AUC, :, 0.847, ), (, all, p<0.05, for, AUC, comparison, ), [, 27, ], .]","[DET, ADJ, NOUN, VERB, SCONJ, DET, NOUN, ADP, VERB, ADJ, ADJ, NOUN, VERB, PROPN, CCONJ, NOUN, SYM, NOUN, NOUN, PUNCT, PROPN, PUNCT, PUNCT, PROPN, PUNCT, NUM, PUNCT, AUX, ADJ, ADP, NOUN, ADP, PRON, ADP, DET, NOUN, ADV, PUNCT, PROPN, PUNCT, NUM, ADP, PROPN, PUNCT, CCONJ, NUM, ADP, PROPN, PUNCT, PUNCT, PRON, NUM, NOUN, AUX, ADJ, ADP, DET, VERB, NOUN, ADP, NUM, ADJ, NOUN, PUNCT, PROPN, PUNCT, NUM, PUNCT, PUNCT, PRON, NUM, ADP, PROPN, NOUN, PUNCT, PUNCT, NUM, PUNCT, PUNCT]","[B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-AC, B-O, B-LF, I-LF, I-LF, I-LF, B-O, B-AC, B-O, B-O, B-AC, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O,

### Set Task
in this project we are doing Named Entity Recognition so I set the task to "ner"

In [9]:
task = "ner"

# Experiment 1 (Model)
HMM vs BERT

## HMM
The following is the implementation of an HMM model

In [2]:
import nltk
from sklearn.metrics import precision_recall_fscore_support

In [4]:
sentences = train[:]["tokens"]
tags = train[:]["ner_tags"]

NameError: name 'train' is not defined

In [ ]:
def replace_numbers_with_tags(array, tags):
  """
  Replaces numbers in a 2D array with corresponding tags from a provided list.

  Args:
      array: A 2D list of numbers (e.g., [[0, 1, 2], [3, 4, 1]]).
      tags: A list of tags corresponding to the numbers in the array (e.g., ['B-O', 'B-AC', 'I-AC', 'B-LF', 'I-LF']).

  Returns:
      A new 2D list with numbers replaced by tags.
  """

#   # Check if the lengths of the array and tags list match
#   if len(tags) != len(set([item for sublist in array for item in sublist])):
#     raise ValueError("Number of tags must match the unique numbers in the array.")

  # Create a dictionary for efficient lookup (number -> tag)
  tag_dict = dict(zip(sorted(set([item for sublist in array for item in sublist])), tags))

  # Replace elements in the array using list comprehension
  return [[tag_dict[num] for num in row] for row in array]

In [ ]:
label_list = dataset["train"].features[f"ner_tags"].feature.names
print(label_list)

In [ ]:
tags = replace_numbers_with_tags(tags, label_list)

In [ ]:
print(f"sentence: {sentences[0]}")
print(f"tags: {tags[0]}")

In [ ]:
char_set = set()
for sentence in sentences:
    for word in sentence:
        for char in word:
            char_set.add(char)
char_set = list(char_set)
print(f"char_set: {char_set}")

In [ ]:
def combine_lists_elementwise(list_A, list_B):
  """
  Combines two 2D lists of strings element-wise into a 2D list of tuples.

  Args:
      list_A: A 2D list of strings (e.g., [['A', 'A', 'A'], ['A', 'A', 'A']]).
      list_B: Another 2D list of strings with the same dimensions as list_A.

  Returns:
      A 2D list of tuples, where each tuple combines corresponding elements from list_A and list_B.

  Raises:
      ValueError: If the dimensions of list_A and list_B don't match.
  """

  # Check if dimensions match
  if len(list_A) != len(list_B) or len(list_A[0]) != len(list_B[0]):
    raise ValueError("Dimensions of lists A and B must be equal.")

  # Create the resulting list using list comprehension
  return [[(a, b) for a, b in zip(row_a, row_b)] for row_a, row_b in zip(list_A, list_B)]

In [ ]:
trainer = nltk.tag.hmm.HiddenMarkovModelTrainer(states=label_list, symbols=char_set)
data = combine_lists_elementwise(sentences.copy(), tags.copy())
print(data[0])

In [ ]:
model = trainer.train_supervised(data)

In [ ]:
import dill

# Assuming you have trained your model (trainer, model, sentences, etc.)

# Open a file for writing in binary mode
with open('hmm_model.dill', 'wb') as f:
  # Dill can handle more complex objects than pickle
  dill.dump(model, f)

print("Model saved as hmm_model.dill")

In [ ]:
# Open the saved model file in binary read mode
with open('hmm_model.dill', 'rb') as f:
  # Load the model back into a variable using dill.load
  model = dill.load(f)

print("Model loaded successfully!")

In [ ]:
test_sentences = dataset["test"][:]["tokens"]
correct_tags = dataset["test"][:]["ner_tags"]
correct_tags = replace_numbers_with_tags(correct_tags, label_list)
predicted = []
for sentence in test_sentences:
    test_result = model.tag(sentence)
    out_tags = []
    for word, tag in test_result:
        out_tags.append(tag)
    predicted.append(out_tags)
predicted[0]

In [ ]:
correct_tags = [item for sublist in correct_tags for item in sublist]
predicted = [item for sublist in predicted for item in sublist]
print(f"number of predictions: {len(predicted)}\nnumber of correct answers: {len(correct_tags)}")

print(correct_tags[:100])
print(predicted[:100])

In [ ]:
precision, recall, f1, support = precision_recall_fscore_support(correct_tags, predicted, labels = label_list)
print(f"precision: {precision}\nrecall: {recall}\nf1-score: {f1}\nsupport: {support}")

## BERT

The following is the implementation of BERT model

### Tokenizer

In [10]:
from transformers import AutoTokenizer
import transformers
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True) # use AutoTokenizer because it defaults to fast tokenizers where as using the BERT Tokenizer does not
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

#### Quick Example

In [11]:
# print an example tokenized text
example = train[0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'alternatively', ',', 'fi', '##bro', '##bla', '##sts', 'were', 'plate', '##d', 'sparsely', 'so', 'that', 'they', 'did', 'not', 'touch', 'each', 'other', 'and', 'induced', 'into', 'qui', '##escence', 'by', 'serum', 'starvation', 'and', 'monitored', 'after', '4', 'd', '(', 'serum', '-', 'starved', 'for', '4', 'd', '[', 'ss', '##4', ']', ')', 'or', '7', 'd', '(', 'serum', '-', 'starved', 'for', '7', 'd', '[', 'ss', '##7', ']', ')', '.', '[SEP]']


In [12]:
# check the length of tokens is the same as in the dataset sample
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])
print(tokenized_input.word_ids())

word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"])) # if it prints the same number twice, then everything is working

[None, 0, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 48, 49, 50, 51, None]
61 61


In [14]:
label_all_tokens = True
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True) ## For some models, you may need to set max_length to approximately 500.

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [15]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

2024-03-25 14:30:35.107266: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 14:30:35.107379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 14:30:35.235482: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


  0%|          | 0/113 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
model_name = model_checkpoint.split("/")[-1]
batch_size = 16
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy ='steps',
    eval_steps = 1000,
    logging_steps = 100,
    save_total_limit = 1,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.001,
    save_steps=1000,
    metric_for_best_model = 'f1',
    load_best_model_at_end=True,
    report_to=['none'],
)

In [18]:
from transformers import DataCollatorForTokenClassification
from datasets import load_metric
data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'AC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'LF': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'O': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 38},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [19]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [20]:
from transformers import EarlyStoppingCallback
BERTtrainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Training

In [22]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [23]:
BERTtrainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1000,0.195400,0.193066,0.939140,0.920015,0.929479,0.923688
2000,0.183400,0.168574,0.944362,0.929060,0.936648,0.931450
3000,0.159300,0.158927,0.947850,0.930396,0.939042,0.933631
4000,0.165900,0.150122,0.947932,0.934383,0.941109,0.936172
5000,0.157900,0.146667,0.949776,0.936980,0.943335,0.938631
6000,0.148900,0.141664,0.950283,0.939457,0.944839,0.940253
7000,0.151000,0.140667,0.951054,0.939505,0.945244,0.940679


TrainOutput(global_step=7041, training_loss=0.17479819706972477, metrics={'train_runtime': 3465.0789, 'train_samples_per_second': 32.511, 'train_steps_per_second': 2.032, 'total_flos': 9829844359441080.0, 'train_loss': 0.17479819706972477, 'epoch': 1.0})

## Evaluation

In [24]:
BERTtrainer.evaluate() ## Final Evaluation on the validation set

{'eval_loss': 0.1406669169664383,
 'eval_precision': 0.9510539780222387,
 'eval_recall': 0.9395051506239975,
 'eval_f1': 0.9452442902572998,
 'eval_accuracy': 0.9406789553624619,
 'eval_runtime': 206.7355,
 'eval_samples_per_second': 116.768,
 'eval_steps_per_second': 29.192,
 'epoch': 1.0}

# Experiment 2 (Tokenization)

# Experiment 3 (Loss Functions)

# Experiment 4 (Hyperparameters)